In [ ]:
# General imports
import numpy as np

# Pre-defined ansatz circuit, operator class and visualization tools
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp
from qiskit.visualization import plot_distribution

# Qiskit Runtime
# from qiskit_ibm_runtime import QiskitRuntimeService
# from qiskit_ibm_runtime import Estimator, Sampler, Session, Options

# SciPy minimizer routine
from scipy.optimize import minimize

# rustworkx graph library
import rustworkx as rx
from rustworkx.visualization import mpl_draw

In [ ]:
# The edge syntax is (start, end, weight)
# edges = [(0, 1, 1), (0, 2, 1), (0, 3, 1), (0, 4, 1)]
# num_nodes = 5
edges = [(0, 1, 1), (1, 2, 1), (2, 3, 1), (3, 0, 1)]
num_nodes = 4

In [ ]:
G = rx.PyGraph()
G.add_nodes_from(range(num_nodes))
G.add_edges_from(edges)
mpl_draw(G, pos=rx.bipartite_layout(G, {0}), with_labels=True, node_color="#EE5396", font_color="#F4F4F4")

In [ ]:
hamiltonian_ibm = SparsePauliOp.from_list([("IIZZ", 1), ("IZZI", 1), ("ZZII", 1), ("ZIIZ", 1)])
ansatz_ibm = QAOAAnsatz(hamiltonian_ibm, reps=2)
ansatz_ibm.decompose(reps=3).draw(output="mpl", style="iqp")

In [ ]:
def cost_func(params, ansatz, hamiltonian, estimator):
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost

In [ ]:
from qiskit.primitives import Estimator, Sampler

estimator = Estimator(options = {"shots": 1024})
sampler = Sampler(options = {"shots": 1024})

In [ ]:
x0 = 2 * np.pi * np.random.rand(ansatz_ibm.num_parameters)
x0

In [ ]:
res = minimize(cost_func, x0, args=(ansatz_ibm, hamiltonian_ibm, estimator), method="COBYLA")
res

In [ ]:
qc = ansatz_ibm.assign_parameters(res.x)
qc.measure_all()
samp_distribution = sampler.run(qc).result().quasi_dists[0]

In [ ]:
plot_distribution(samp_distribution.binary_probabilities())